# Intro to RL 

## Problem setup: We want to make an AI that is able to complete a simple video game.

### What is the game we are going to start with?
In this game, we want our agent (character) to move through the 2D world and reach the goal. At each timestep our agent can to either move up, down, left or right. The agent cannot move into obstacles, and when it reaches the goal, the game ends.

![](./images/griddy.gif)

We are going to use an environment that we built, called Griddy, that works in exactly the same way as other environments provided as part of openAI gym. 


The main ideas are:
<ul>
<li>we need to create our environment</li>
<li>we need to initialise it by calling `env.reset()`</li>
<li>we can increment the simulation by one timestep by calling `env.step(action)`</li>
</ul>

Check out [openAI gym's docs](http://gym.openai.com/docs/) to see how the environments work in general and in more detail.

Let's set up our simulation to train our agent in.


In [1]:
# IMPORTS
import time
import pickle
import numpy as np

from GriddyEnv import GriddyEnv # make sure you: pip3 install GriddyEnv

# SET UP THE ENVIRONMENT
env =     ## create the environment

SyntaxError: invalid syntax (<ipython-input-1-c5be1fa42853>, line 9)

## Once we have an agent in the game, what do we do?

Our agent has no idea of how to win the game. 
It simply observes states, and takes actions.
As a result of these actions, the agent will see the environment change to a new state and also receive some sensation. This sensation, which may be good or bad, is called a **reward**.
and then  that change based on it's actions and receives a reward signal for doing so.

This continuous interaction between the agent and our environment sets up the framework for a **reinforcement learning problem**, in an agent-environment loop as shown below.

![](images/agent-env-loop.png)

So without any prior knowledge, the agent has to learn about the game for itself. Just like a baby learns to interact with it's world by playing with it, our agent has to try random actions in the environment to figure out what causes it to receive negative or positive rewards.

A function which tells the agent what actions to take from a given state is called a **policy**.

![](./images/policy.png)

Policies can be deterministic or stochastic (have randomness).

Mathematically, a policy is a probability distribution over actions, conditioned on the state. 

## Defining our RL problem

In our case:

### Action Space
The action space consists of 4 unique actions: 0, 1, 2, 3<br>
0 - Move left<br>
1 - Move right<br>
2 - Move up<br>
3 - Move down<br>

### Observation Space
Has shape (3, 4, 4). Our grid world is 4x4<br>
Each of the 3 channels is a binary mask for the location of different objects within the environment.<br>
Channel 0 - Goal<br>
Channel 1 - Wall<br>
Channel 2 - Agent<br>
This is what our environment returns us on a state transition.

I've simplified the code by including lines that convert this tensor state representation to an integer. This better represents the size of our environment and makes visualisation easier.

Let's drop our agent into the environment, implement a random policy, and then watch it act for a few episodes.

In [ ]:
#Visualise agent function
def visualise_agent(policy, n=5):
    try:
        for trial_i in range(n):
            observation = env.reset()
            done=False
            t = 0
            while not done:
                env.render()
                agent_pos = list(zip(*np.where(observation[2] == 1)))[0]
                state = 4 * agent_pos[0] + agent_pos[1]
                policy_action = policy(state)
#                 print('policy_action:', policy_action)
                observation, reward, done, info = env.step(policy_action)
                time.sleep(0.1)
                t += 1
            env.render()
            time.sleep(1.5)
            print("Episode {} finished after {} timesteps".format(trial_i, t))
        env.close()
    except KeyboardInterrupt:
        env.close()

In [ ]:
# IMPLEMENT A RANDOM POLICY
def random_policy(state):
    action = ##
    return action

In [ ]:
visualise_agent(random_policy)

## How do we know if we are doing well?

When our agent takes this action and moves into a new state, the environment returns it a reward. The reward when it reaches the goal is +1, and 0 everywhere else. The reward that the agent receives at any point can be considered as what it feels in that moment - like pain or pleasure.

**However**, the reward doesn't tell the agent how good that move actually was, only whether it sensed anything, and how good or bad that sensation was at that particular moment.

E.g.
- Our agent might not receive any reward for stepping toward the goal, even though this might be a good move.
- A robot might receive a negative reward as it's battery depletes, but still make good progress towards its goal.
- A chess playing agent might receive a positive reward for taking an opponent's piece, but make a bad move in doing so by exposing its king to an attack eventually causing it to lose the game.

What we really want to know is not the instantaneous reward, but "How good is the position I'm in right now?", that is, what amount of reward can our agent get from this point onwards.
This future reward is also known as the return.

![](./images/undiscounted_return.jpg)

#### Is getting a reward now as good as getting the same reward later?
- What if the reward is removed from the game in the next timestep?
- Would you rather be rich now or later?
- What if a larger reward is introduced and you don't have enough energy to reach both?
- What about inflation?

It's better to get rewards sooner rather than later.

![](./images/decay.jpg)

We can encode this into our goal by using a **discount factor**, $\gamma \in [0, 1]$ ($\gamma$ between 0 and 1). 
The discount factor is the coefficient of a reward $t$ timesteps in the future, raised to the power of $t$. Because it is less than 1, raising to the power reduces its value. As such, this coefficient weights rewards further away in the future by a lesser number than those nearby in time.

This makes our agent value more immediate rewards more than those which can be reached further in the future.
This makes the goal become:

![](./images/discounted_return.png)

This is called the **discounted return**. From here on, when we say return, we mean discounted return unless otherwise specified, because we rarely use the undiscounted version.

Once an agent has played a whole game it can calculate the return for each state it visited by simply adding up the discounted reward it achieved from there on.

The value of these return values can also be defined recursively as shown below.

![](./images/recursive_return.png)
 
Because of this recursive relationship, we can calculate the experienced returns for each visited state in a single pass through the trajectory of that episode. We do this by working backwards, firstly calculating the  return from the terminal state (always zero), and then recursively calculating the return from the previous timestep by discounting it and adding the reward from that timestep.
This process is called ***backup***.

The return of a terminal state is always zero, because from there the episode will have terminated and hence the agent will not be able to attain any further reward.

![](./images/backup.png)


## Value functions - so how good is each state?

Now we know how to calculate the return that we experienced during any single episode. 
But this is just a single sample estimate. 
In general, the goal of reinforcement learning is to maximise the **expected** future reward. 
That is, to maximise the average return from a the current state onwards. 
This quantity is defined as the **state-value** of a state, commonly just referred to as the  or **value** of a state. 
A function that returns this value given a state is called a value function.

![](./images/value_def.jpg)

**Note that a value function must correspond to some policy.** If we follow a bad policy then states will have lower values than if we follow a good policy. If we change the policy, then the value function will change

### The Bellman equation for $V(S)$

Using the recursive definition of the return, we can express the value function recursively.

The Bellman equations are those which express value functions recursively.

![](./images/value-bellman.jpg)

#### Recovering the Bellman optimality question for $V(S)$

![](./images/value-optim-deriv.jpg)


### The Bellman Optimality equation for $V(S)$

For a value function to be better than some other, it must have values greater than or equal to those for the other value function for all states.

If we are acting optimally at any given state, then the action we take should consider all actions and then take the one with the best expected return. This return is calculated using the value for the next state.

![](./images/value-optim.jpg)

## How can we use these tools, and improve our agent's performance?

### Method 1: Dynamic Programming (DP) methods for computing value functions and improving policies

The term dynamic programming (DP) refers to a collection of algorithms that can be used to compute optimal policies given a perfect **model** of the environment.

A model tells us how the environment will change when we take certain actions. It may be stochastic or deterministic. A model can allow us to **simulate** the progression of the environment, taking any action from any state.

![](./images/model.jpg)

**A model will also be referred to as a transition function.**

Fortunately (by our design) in this simple version of the game, we do know exactly what actions will lead us to what states. That means we have a perfect **model** of the environment. A model is a function that tells us how the state will change when we take certain actions. E.g. we know that if the agent tries to move up into an empty space, then that's where it will end up.

Let's run the cell below to define our transition function

In [ ]:
# TRANSITION FUNCTION/ENVIRONMENT MODEL
def transition(state, action):
    LEFT = 0
    DOWN = 1
    RIGHT = 2
    UP = 3
    
    GOAL = 15
    nrow = 4
    ncol = 4
    
    if state == GOAL: #they have already reached the goal and need to reset the environment
        reward=0
        done=1
        return state, reward, done
    
    row = state // 4 # nearest factor of nrows
    col = state % 4 # remainder of ncols
    if action == LEFT:
        new_state = (row, max(col-1, 0))
    if action == DOWN:
        new_state = (min(row+1, nrow - 1), col)
    if action == RIGHT:
        new_state = (row, min(col+1, ncol - 1))
    if action == UP:
        new_state = (max(row-1, 0), col)
    new_state = nrow * new_state[0] + new_state[1] # convert back to integer
    reward = 0
    done=0
    if new_state == GOAL: #they have just reached the goal state
        reward = 1
        done=1
    return new_state, reward, done

If we have a model, we can look ahead to the successor states reachable from our current state.
If we also had a way to estimate the value function, then we could look ahead to the state that each action would take us to and take the action which results in the best expected return.

Acting greedily with respect to a value function for an optimal policy will produce optimal behaviour.

![](./images/follow_values.jpg)

#### Computing value functions using dynamic programming
## Algorithm 1: Policy iteration

### Step 1: Policy evaluation step

Policy evaluation is the process of approximately evaluating the value function for our current policy.

How can we do this using our environment model?

The Bellman equations define the value of any state recursively, as a function of it successor state.
We know that we can use our model to simulate the next states that our agent will move into by taking any given action, and that it defines what rewards we might receive.
Given this, along with the fact that our value table already contains estimates

### Step 2: Policy improvement step

Policy improvement is done by setting the new policy to be greedy with respect to the value function. This means that the new policy will consider all actions,and then take the action that leads to the greatest expected return, based on our current value function.

### Full policy iteration algorithm

The full policy iteration algorithm iterates between policy evaluation and policy improvement. This alternatively improves the policy by making it greedy with respect to the value function, and then improves the value function by minimising the Bellman error.

Let's put these two steps together to produce the full policy iteration algorithm.

![](./images/policy-iteration.jpg)

### Does this converge to an optimal policy?

For a policy $\pi'$ to better than some other policy $\pi$?, it must be such that $v_{\pi'}(s) \geq v_{\pi}(s)$ for all states.

![](./images/convergence.jpg)

In [ ]:
def initialise_value_table(num_states=16):
    value_table = {} # start off with empty map
    for s in range(num_states): # for each state
        value_table[s] = np.random.rand()
    return value_table

def initialise_deterministic_policy(num_states=16, num_actions=4):
    policy = {} # start off with empty map
    for s in range(num_states):
        policy[s] = ##choose random action
    return policy

In [ ]:
# COMPUTING VALUE FUNCTION USING DYNAMIC PROGRAMMING
def policy_evaluation(policy, value_table, discount_factor, error_threshold=0.01, num_states=16):
    print()
    print(value_table)
    new_value_table = value_table # init new value table to be filled in and returned
    converged = False # initially we have not found a converged value function for this policy
    k = 0 # sweep index
    while not converged: # until the value function converges
        print('sweep ', k)
        k += 1 # increment sweep counter
        
        worst_delta = 0 # difference between previous values and iterated values
        for state in range(num_states): # loop over each state
            action =  ## get the action according to current policy
            new_state, reward done, =  ## use model to simulate next state and reward
            if not done: 
                new_val =  ## compute new value for non-terminal state
            else:
                new_val =  ## compute new value for terminal state
            new_value_table[state] =  ## update value table
            delta =  ## find the absolute diff between new val and old val for this state

            # CHECK WORST VALUE FUNCTION ERROR
            if delta > worst_delta: # is this state the one for which our value table is most wrong?
                worst_delta = delta # update worst error for this sweep
                print('worst_delta:', worst_delta)
                
        # CHECK CONVERGED
        if worst_delta < error_threshold: # once the values stop changing
            converged = ## we have found the value function
            print('Converged on value function')
        value_table =  # update value table (took me ages to realise i was missing this line and debug )
    return new_value_table # return converged value table evaluated for this policy
        

In [ ]:
# IMPROVE POLICY
def policy_improvement(value_table, discount_factor): # set a greedy policy which will always be better than the previous
    new_policy = {} # initialise empty new policy to be filled and returned
    action_space = range(4) # 
    for  ## loop over each state
        best_value =  ## initialise best value as negative infinity
        best_action =  ## no best action found yet
        for : ## set the policy as greedy with respect to the value function
            new_state, reward = ## use model to simulate next state and reward
            if not done: 
                value =  ## compute value for non-terminal state
            else:
                value =  ## compute value for terminal state
            
            # CHECK MAX VALUE
            if value > best_value: # checking all actions, which gives this state the best value?
                best_value = value # update best value
                best_action = action # update best action found so far
                
        # SET GREEDY POLICY
        new_policy[state] =  ## update new policy to take best action found when it sees this state
    return new_policy

def check_stable_policy(old_policy, new_policy):
    stable = True
    # CHECK STABLE POLICY
    return stable

In [ ]:
# POLICY ITERATION ALGORITHM
def policy_iteration(discount_factor=0.9):
    value_table = ## initiailise value table
    policy = ## initialise deterministic policy
    policy_stable = ## initialise false
    policy_idx = 0 # outer to check how many policies we've tried
    while not policy_stable: # until convergence
        
        # POLICY ITERATION
        print('Evaluating policy ', policy_idx)
        value_table =  ## converge on value function
        
        # POLICY IMPROVEMENT
        print('Iterating policy ', policy_idx)
        new_policy =  ## get greedy policy using converged value function
        
        # CHECK CONVERGENCE
        if check_stable_policy(policy, new_policy): # compare policies
            policy_stable = True # when they have the same greedy action for each state in the state space
            print('Policy now stable - optimal policy found')
            
        policy =  # update policy
        
    print('Optimal policy:', policy)
    return policy, value_table
        
optimal_policy, optimal_value = policy_iteration()

In [ ]:
def policy_map_to_func(policy_map): # turn our dict into a function
    def policy(k):
        return policy_map[k]    
    return policy

optimal_policy_func = policy_map_to_func(optimal_policy)
visualise_agent(optimal_policy_func)

Notice that the output policy is optimal, but it is not the only optimal policy. For this map it's easy to see that there could be alternatives

## Algorithm 2: Value iteration - forget the explicit policy

Value iteration is very similar to policy iteration - but we only perform a single sweep over the state space when updating our value function, instead of repeating this until our approximate value function converges for the current policy.

The policy evaluation phase requires us to sweep over the entire state space repeatedly. If the state space is large, then this can be computationally expensive.

It also turns out that it's not necessary.

Every time we perform a sweep, the value function gets closer to the true value function for the current policy. 
It can be seen that improved policies are found by greedily following value functions that result from truncated policy evaluations.

Additionally, if we are greedily following the value function rather than querying the policy each timestep then we don't need to represent the policy explicitly. 
 

Notice that in this case, to learn an optimal policy, we never have to represent it explicitly. There is no function which represents the policy. Instead we just look-ahead and choose the action that maximises the value of the next state.


### Where does value iteration come from?

In the first line below, the maximum state-value of a state is equivalent to the maximum action-value when taking the best action in that state. Following this, we can derive a recursive definition of the optimal value function.

In the last step, we even remove the policy from the equation entirely! This means that value iteration never needs to explicitly represent a policy in terms of a function that takes in a state and returns a distribution over actions.
Instead, value iteration uses a **model**, $p(s', r | s, a)$, to look one step ahead, and take the action, $a$, that most likely leads it to the next state that has the best state-value function.

A **model** defines how the state changes. It is also known as the transition dynamics of the environment. In our case the model is really simple: we are certain that taking the action to move right will move our agent one space to the right as long as there are no obstacles. There is no randomness in our environment (e.g. no wind that might push us into a different cell when we try to move right). That is, our environment is deterministic, not stochastic.

![](./images/bellman_op_v.png)

![](./images/backup_v.png)

![](./images/update_rule_v.png)  

Let's now implement value iteration

In [ ]:
def value_iteration(num_states=16, num_actions=4, error_threshold=0.01, discount_factor=0.9):
    converged = False
    value_table = initialise_value_table()
    while not converged:
        new_value_table = {}
        
        # POLICY EVALUATION WITH CONSTANT IMPLICIT POLICY UPDATES
        worst_delta = 0
        for state in range(num_states): # sweep once over state space
            
            # FIND BEST ACTION FOR THIS STATE BASED ON CURRENT VALUE TABLE
            best_action = None
            best_value = -float('inf')
            for action in range(num_actions): # max over actions
                new_state, reward, done = ## simulate next state
                if not done: 
                    new_val =  ## compute new value for non-terminal state
                else:
                    new_val =  ## compute new value for terminal state
                if new_val > best_value:
                    best_action = action
                    best_value = new_val
            
            # CHECK ERROR
            delta = ## check value change for each state
            worst_delta = max(delta, worst_delta)
            
            new_value_table[state] = best_value # update value table greedily

        value_table = new_value_table
        
        # NOTE THAT THERE IS NO POLICY IMPROVEMENT STEP - IT'S DONE IMPLICITLY BY MAXIMISING VALUES OVER ACTIONS
        
        # CHECK CONVERGENCE
        if worst_delta < : # check if worst delta below threshold
            print('Converged')
            converged = True
    
    # RETURN DETERMINISTIC POLICY - at this point, the algorithm has completed its job
    # now the value function should be optimal, and hence correspond to an optimal policy which we can extract
    output_policy = {}
    for state in range(num_states):    
        best_action = None
        best_value = -float('inf')
        for action in range(num_actions): # max over actions
            new_state, reward, done = ##
            if not done: 
                new_val =  ## compute new value for non-terminal state
            else:
                new_val =  ## compute new value for terminal state
            if new_val > best_value:
                best_action = action
                best_value = new_val
        output_policy[state] = best_action
    return output_policy

optimal_policy = value_iteration()
optimal_policy_func = policy_map_to_func(optimal_policy)
visualise_agent(optimal_policy_func)


Value iteration and policy iteration are types of **value based** method - they use a value or Q function to find an optimal policy.

### Drawbacks of Dynamic Programming
#### What if we aren't fortunate enough to have a model?
We very rarely have access to a transition model for the environment.

#### Policy evaluation requires sweeping over entire states

Backgammon has over $10^{20}$ states. Performing a sweep over this many states is extremely computationally expensive. 
Some of these states are very rare. Some might never actually be seen through experience.
How can we continue to evaluate policies without needing this sweep?


## End of notebook!

Next you might want to check out:
- [Q Learning](https://github.com/AI-Core/Reinforcement-Learning/blob/master/Q%20Learning.ipynb)
